
# Advent of Code 2022

> Raise your quality standards as high as you can live with, avoid wasting your time on routine problems, and always try to work as closely as possible at the boundary of your abilities. Do this, because it is the only way of discovering how that boundary should be moved forward.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
#type: ignore
from functools import reduce, lru_cache
from operator import mul, and_
from dataclasses import dataclass
from collections import defaultdict, Counter
from enum import Enum
from typing import List


def prod(u): 
    """Older Pythons don't have this."""
    return reduce(mul, u)


def inputfunc(day, kind='lines', testing=False):
    """Generator to read input files."""
    filename = 'test.txt' if testing else f"input/{day}.txt"

    def gen(func):
        if kind == 'lines':
            text = [x.strip() for x in open(filename)]
        elif kind == 'chunks':
            text = [
                x.strip()
                for x in open(filename).read().split('\n\n')
                if x.strip()
            ]
        elif kind == 'single':
            text = open(filename).read().strip()
        elif kind == 'commas':
            text = [x.strip() for x in open(filename).read().split(',')]
        elif kind == 'raw':
            text = open(filename).read()

        def inner():
            return func(f=text)
        return inner
    return gen

## [Day 1 - Calorie Counting](https://adventofcode.com/2022/day/1)

In [2]:
@inputfunc(1, kind='chunks')
def input_1(f):
    return ([int(j) for j in l.split('\n')] for l in f)


A = max(sum(l) for l in input_1())
assert A == 68467


A = sum(sorted((sum(l) for l in input_1()), reverse=True)[:3])
assert A == 203420

## [Day 2 - Rock Paper Scissors](https://adventofcode.com/2022/day/2)

In [3]:
@inputfunc(2)
def input_2(f):
    return (l.split() for l in f)


class Play(Enum):
    ROCK = 1
    PAPER = 2
    SCISSORS = 3

    def __gt__(self, other):
        return (self.value - other.value) % 3 == 1


def score_all(l):
    def score_play(other, me):
        other_play = {'A': Play.ROCK, 'B': Play.PAPER, 'C': Play.SCISSORS}[other]
        my_play = {'X': Play.ROCK, 'Y': Play.PAPER, 'Z': Play.SCISSORS}[me]
        if my_play > other_play:
            return 6 + my_play.value
        elif my_play == other_play:
            return 3 + my_play.value
        else:
            return my_play.value

    return sum(score_play(other, me) for other, me in l)


def score_strat(l):
    def score_play(other, me):
        other_play = {'A': Play.ROCK, 'B': Play.PAPER, 'C': Play.SCISSORS}[other]
        adj = {'X': -1, 'Y': 0, 'Z': 1}[me]
        my_play = next(x for x in Play if (x.value % 3) == (other_play.value + adj) % 3)
        return my_play.value + 3 * (adj + 1)
    
    return sum(score_play(other, me) for other, me in l)


A = score_all(input_2())
assert A == 15632


A = score_strat(input_2())
assert A == 14416

## [Day 3 - Rucksack Reorganization](https://adventofcode.com/2022/day/3)

In [4]:
@inputfunc(3)
def input_3(f):
    return f


def prio(c):
    if c.isupper():
        return 27 + ord(c) - ord('A')
    else:
        return 1 + ord(c) - ord('a')


def common_split(l):
    def find_common(s):
        mid = len(s) // 2
        return (set(s[:mid]) & set(s[mid:])).pop()

    return sum(prio(find_common(s)) for s in l)


def common_triples(l):
    def find_common(ss):
        return (reduce(and_, ss)).pop()

    def split(ll, k):
        t = [None for _ in range(k)]
        for n, i in enumerate(ll):
            t[n % k] = i
            if n % k == k - 1:
                yield t

    return sum(prio(find_common(set(s) for s in ss)) for ss in split(l, 3))


A = common_split(input_3())
assert A == 7581


A = common_triples(input_3())
assert A == 2525

## [Day 4 - Camp Cleanup](https://adventofcode.com/2022/day/4)

In [5]:
@inputfunc(4)
def input_4(f):
    def parse(l):
        a, b = l.split(',')
        al, ah = a.split('-')
        bl, bh = b.split('-')
        return (int(al), int(ah)), (int(bl), int(bh))

    return (parse(l) for l in f)


def fully_contains(a, b):
    al, ah = a
    bl, bh = b
    return (al <= bl <= bh <= ah) or (bl <= al <= ah <= bh)


def disjoint(a, b):
    al, ah = a
    bl, bh = b
    return (al <= ah < bl <= bh) or (bl <= bh < al <= ah)


A = sum(1 for a, b in input_4() if fully_contains(a, b))
assert A == 511


A = sum(1 for a, b in input_4() if not disjoint(a, b))
assert A == 821

## [Day 5 - Supply Stacks](https://adventofcode.com/2022/day/5)

In [6]:
@dataclass
class Move:
    qty: int
    ori: int
    dest: int


class State:
    def __init__(self, stacks):
        self.stacks = stacks

    def execute(self, i: Move, mode: str = 'lifo'):
        self.stacks[i.dest] += self.stacks[i.ori][-i.qty:][::-1 if mode == 'lifo' else 1]
        self.stacks[i.ori] = self.stacks[i.ori][:-i.qty]


@inputfunc(5, kind='raw')
def input_5(f):
    state, instr = f.split('\n\n')

    # Decode initial state
    B = []
    for l in state.split('\n'):
        if not '[' in l:
            break
        B.append([l[i] for i in range(1, len(l), 4)])

    stacks = defaultdict(list)
    for l in B:
        for n, x in enumerate(l, 1):
            if x != ' ':
                stacks[n].append(x)

    S = State({k: list(reversed(v)) for k, v in stacks.items()})

    # Decode instruction list
    M = []
    for l in instr.split('\n'):
        s = (l.strip()
            .replace('move', '')
            .replace('from', '')
            .replace('to', '')
            .split())
        if len(s) == 3:
            M.append(Move(*(int(c) for c in s)))

    return S, M


def get_final_state(state: State, moves: List[Move], mode='lifo'):
    for m in moves:
        state.execute(m, mode)
    return ''.join(state.stacks[k][-1] for k in sorted(state.stacks))


state, moves = input_5()
A = get_final_state(state, moves, mode='lifo')
assert A == 'CNSZFDVLJ'

state, moves = input_5()
A = get_final_state(state, moves, mode='fifo')
assert A == 'QNDWLMGNS'

## [Day 6 - Tuning Trouble](https://adventofcode.com/2022/day/6)

In [7]:
@inputfunc(6, kind='single')
def input_6(f):
    return f


def first_diff(s, n):
    cnt = Counter()

    for i, c in enumerate(s, 1):
        cnt[c] += 1
        if i > n:
            top = s[i - n - 1]
            cnt[top] -= 1
            if cnt[top] == 0:
                del cnt[top]
        if len(cnt) == n:
            return i


A = first_diff(input_6(), 4)
assert A == 1920


A = first_diff(input_6(), 14)
assert A == 2334

## [Day 7 - No space left on device](https://adventofcode.com/2022/day/7)

In [8]:
@dataclass
class File:
    name: str
    size: int


@dataclass
class Directory:
    name: str
    subdirs: 'List[Directory]'
    files: List[File]
    parent: 'Directory | None'

    def recursive_size(self):
        return sum(f.size for f in self.files) \
            + sum(d.recursive_size() for d in self.subdirs)


@inputfunc(7)
def input_7(f):
    commands = []
    contents = []

    for l in f:
        t = l.split()
        if t[0:2] == ['$', 'cd']:
            commands.extend(contents)
            contents = []
            commands.append(('move', t[2]))
        elif t[0:2] == ['$', 'ls']:
            pass
        elif t[0] == 'dir':
            contents.append(('putdir', t[1]))
        else:
            try:
                contents.append(('putfile', (t[1], int(t[0]))))
            except ValueError:
                pass

    commands.extend(contents)
    return commands


def build_structure(commands):
    root = Directory('/', [], [], None)
    cd = None

    for what, arg in commands:
        if (what, arg) == ('move', '/'):
            cd = root
        elif (what, arg) == ('move', '..'):
           cd = cd.parent
        elif what == 'move':
            l = [d for d in cd.subdirs if d.name == arg]
            if l:
                cd = l[0]
            else:
                nd = Directory(arg, [], [], cd)
                cd.subdirs.append(nd)
        elif what == 'putdir':
            cd.subdirs.append(Directory(arg, [], [], cd))
        elif what == 'putfile':
            cd.files.append(File(*arg))

    return root


def sum_directory_size(root: Directory, maxsize: int):
    own = root.recursive_size()
    capped = own if own <= maxsize else 0
    return capped + sum(sum_directory_size(d, maxsize) for d in root.subdirs)


def minimum_size_above(root: Directory, minsize: int):
    m = (minimum_size_above(d, minsize) for d in root.subdirs)
    return min((x for x in (*m, root.recursive_size()) if x >= minsize), default=10**100)


R = build_structure(input_7())

A = sum_directory_size(R, 100_000)
assert A == 1391690

A = minimum_size_above(R, R.recursive_size() - 40_000_000)
assert A == 5469168